In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# Для данной модели путь понадобится изменить в ячейках ниже

# Модель RuDalle

In [ ]:
!pip install rudalle==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Начинаем генерировать

In [ ]:
# Здесь нужно изменить первую директорию в аргументах функции cp:

!mkdir /content/checkpoints
!mkdir /content/gen_images
!cp /content/gdrive/MyDrive/ru-dalle_12ep.pt  /content/checkpoints/ru-dalle_12ep.pt
                                                        # выше вместо первого аргумента укажите путь до загруженных
                                                        # на гугл-диск весов модели

mkdir: cannot create directory ‘/content/checkpoints’: File exists
mkdir: cannot create directory ‘/content/gen_images’: File exists


In [ ]:
from rudalle.pipelines import generate_images, show, super_resolution, cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae, get_realesrgan, get_ruclip
from rudalle.utils import seed_everything
import os
import multiprocessing
import torch
from psutil import virtual_memory


device = 'cuda'
model_path = os.path.join('/content/checkpoints/ru-dalle_12ep.pt') # здесь тоже может понадобиться изменить путь (если вы меняли имя файла)
model = get_rudalle_model('Malevich', pretrained=True, fp16=True, device=device)
model.load_state_dict(torch.load(model_path)) 
print(f'Loaded from {model_path}')
vae = get_vae().to(device)
tokenizer = get_tokenizer()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
pil_images = []
scores = []
images_num = 3
text = 'Покемон'

img_count = 0
for top_k, top_p in [
        (2048, 0.995),
        (1536, 0.99),
        (1024, 0.99),
        (1024, 0.98),
        (512, 0.97),
        (384, 0.96),
        (256, 0.95),
        (128, 0.95), 
    ]:
      _pil_images, _scores = generate_images(text, tokenizer, model, vae, top_k=top_k, images_num=images_num, top_p=top_p)
      for images in _pil_images:
            images.save(f"gen_images/{img_count:03d}.png")
            img_count += 1
      show(_pil_images, 3, size=28)
      pil_images += _pil_images
show([pil_image for pil_image in pil_images], 3, size=56)

NameError: ignored